In [14]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
#0 prepare data
bc = datasets.load_breast_cancer()
X,Y = bc.data, bc.target
n_samples,n_features=X.shape

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.1, random_state=1234)
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
Y_train=torch.from_numpy(Y_train.astype(np.float32))
Y_test=torch.from_numpy(Y_test.astype(np.float32))

Y_train=Y_train.view(Y_train.shape[0],1)
Y_test=Y_test.view(Y_test.shape[0],1)

In [16]:
#1) Model
# f = w*x+b, sigmoid at the end
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression,self).__init__()
    self.linear = nn.Linear(n_input_features,1)
  def forward(self,x):
    Y_pred = torch.sigmoid(self.linear(x))
    return Y_pred

model = LogisticRegression(n_features)

In [17]:
# 2)loss and optimizer
criterion = nn.BCELoss()
l_rate=0.01
optimizer = torch.optim.SGD(model.parameters(), lr=l_rate)

In [18]:
#3)training loop
n_epoch=1000
for epoch in range(n_epoch):
  #forward pass loss
  Y_pred = model(X_test)
  loss = criterion(Y_pred, Y_test)

  #backward pass
  loss.backward()
  #update
  optimizer.step()
  optimizer.zero_grad()

  if (epoch+1)%100==0:
    print(f'itration: {epoch+1} ,loss = {loss.item():.4f}')
with torch.no_grad():
  Y_predicted = model(X_test) 
  Y_predicted_cls = Y_predicted.round()
  acc = Y_predicted_cls.eq(Y_test).sum()/float(Y_test.shape[0])
  print(f'accuracy = {acc:.4f}')


itration: 100 ,loss = 0.2836
itration: 200 ,loss = 0.2071
itration: 300 ,loss = 0.1721
itration: 400 ,loss = 0.1512
itration: 500 ,loss = 0.1371
itration: 600 ,loss = 0.1267
itration: 700 ,loss = 0.1187
itration: 800 ,loss = 0.1122
itration: 900 ,loss = 0.1069
itration: 1000 ,loss = 0.1024
accuracy = 0.9825
